# Incontro ISLAB 2017-03-14

## Boolean Retrieval

La necessità di avere uno spazio metrico è dovuta alla possibilità che ne consegue di verificare la similarità.
In questo modo possiamo fare retrieval dei documenti una volta sviluppato il modello.

Una volta definito il modello, dobbiamo preoccuparci di implementarlo:

- in maniera efficiente
- in modo scalabile

Attraverso i sistemi di information retrieval booleano verranno introdotti i concetti fondamentali.

I tre obiettivi dell'information retrieval sono:

- searching
- classification
- data analysis

Matrice termine di documento (**index matrix**), es.

docID | id_tag_1 | id_tag_2 | id_tag_3
------|----------|----------|---------
123   |    1     |   0      |   0
345   |    0     |   1      |   0

I vantaggi di una forma di questo tipo sono:

- facilità nel fare query: e.g. testi delle canzoni taggate peace AND love AND NOT war
    - sono simili i documenti che rispondono alle stesse query
- abbiamo  documenti espressi intermini di terminologia e la terminologia in termini di documenti
    - in uno schema relazionale avrei un problema di dimensioni (ad ogni documento aggiungo colonne)
    - la matrice è molto sparsa (molte celle con val 0)
        - la varietà è data da una questione lessicale (nel nostro caso, non è eccessivamente sparsa perché i tag sono forniti da clarifai)
    
#### Inverted index:

given 0,D,S as the number of token **occurences**, the **size of dictionary** (e.g. number of unique tokens/tags), and the **size of corpus** (e.g. number of documents), respectively, the **sparseness** _S_ of a corpus can be calculated as:


$$ S = \frac {O} {D*S} $$



corpus | O | D | S | _S_
--------|---|---|---|---
gozzano | 17597 | 5189 | 117 | 0.028985
songs | 261273 | 22367 | 4311 | 0.002710


La varietà lessicale della lingua non è infinità, quindi la _sparseness_ non crescerà linearmente.

Per non salvarsi tutti gli 1, si utilizza l'inverted index

Dictionary | Posting
------|----------
123   |    1 23 ..    
345   |    0 45 ..   


```js
{
    "token": "love",
    "posting": [123, 456, 789, ...]
}

```

Pro:

- la cardinalità è pari al dizionario
- indice primario sul token (facilità di accesso)
- usa btree per l'accesso 
    - consente di avere un tempo di accesso dei dati ordinati più efficiente. Consiste in un albero balanced: ad ogni nodo ho sempre lo stesso numero di elementi. Il campo di ricerca _O(k)_ dove k è la dimensione di ogni livello di nodi ). Se uno può scegliere, tende a farlo binario per ragioni di bilanciamento ma è strettamente legato all'allocazione della memoria.

```js
invertedindex.find({"token": "love"})
```

Contro:

- indicizzare è sempre costoso: dipende se si vuole pagare in fase di acquisizione o in fase di query
- limitatamente adottabile su un relazionale
- la dimensione orizzontale può crescere ad libitum
- non sono facilmente distribuibili

#### DB Storage: entry as tuple

```js
{
  "token": "love",
  "posting": 598
},
{
  "token": "love",
  "posting": 789
},
```

Pro:
- la dimensione orizzontale è fissa (quasi relazionale)
- costa pochissimo aggiunta/rimozione/aggiornamento degli elementi
- sort dei posting al query time

Contro:
- la cardinalità cresce notevolmente
- sort dei posting potrebbe essere inefficiente (richiede un secondary index importante)

```js
invertedindex.find({"token": "love"}).sort({"posting":1})
```

Indici primari sparsi: costruiti sulle chiavi primarie dei database. 
 - sparso: Un campo ordinato su disco. Essendo ordinati, mi basta memorizzarne uno solo per capire quale blocco di disco devo caricare in memoria
Indice secondario: solo su token -> non è ordinato e non è univoco


Una index matrix booleana non permette di capire quanto sia significativo un tag rispetto a una risorsa.
Per indicare la significatività di un token posso adattare la mia struttura, eg.

```js
{
   "token": "love",
   "posting": 599,
   "occurences": 3,
   "doc_len": 232
}
```


term frequency| example
---|---
binary | 0,1
row frequency | o(t,d) of t in d
log frequency | 1 + log o(t,d)
normalized frequency | o(t,d) / abs(d)
log normalized frequency | log (1 + o(t,d) / abs(d) )
double normalized frequency | K + (1-K) o(t,d) / max[t' in d] o(t',d)



occurencies per token in document d: o(t, d)

document frequency permette di vedere quanto è diffuso un token.
di solito si considera la inverse document frequency: n docs / docs containing t
N +1 / n + 1

più è alto l'IDF, maggiore è la significatività dei termini

al crescere del numero di occorrenze, la crescita cumulativa dei token è un ramo di iperbole crescente
solitamente le parole con un IDF alto hanno un numero di occrenze minori

L'intersezione è più efficace da determinare in base alla dimensione delle posting list

Come si valuta un sistema di information retrieval:
- restituire risultati giusti (precision P)
- restituire tutti i risultati (recall R)

$$ P = \frac{\left | R\bigcap  E\right |}{\left | R \right |} $$
$$ R = \frac{\left | R\bigcap  E\right |}{\left | E \right |} $$

In [ ]:
import numpy as np

docs_len = [('d671')]
docs = [x[0] for x in docs_len]
words = []
M = np.array()
# transpose M.T
